# Doing a convolution 

# Generate images

In [4]:
# Need to reorder the data for tinto
import pandas as pd

data = pd.read_csv('DataSet/dataWnmhc.csv')
data = data[['datetimestamp', 'T' ,'RH', 'AH', 
             'PT08.S1(CO)', 'PT08.S2(NMHC)', 'PT08.S3(NOx)', 
             'PT08.S4(NO2)', 'PT08.S5(O3)', 'CO(GT)']]
data.to_csv('DataSet/COdata.csv')

In [5]:
from data2Image.models import tinto
model = tinto(blur=True)

In [7]:
model.generateImages('DataSet/COdata.csv', 'imgs/')a

2 4


# Results method

In [50]:
results = pd.DataFrame(columns=['Model', 'mse', 'mae', 'rmse', 'mape', 'r2'])
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import root_mean_squared_error
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics import r2_score


def appendError(model, testy, y_pred):
    model_name = str(type(model).__name__)
    mse = mean_squared_error(testy, y_pred)
    mae = mean_absolute_error(testy, y_pred)
    rmse = root_mean_squared_error(testy, y_pred)
    mape = mean_absolute_percentage_error(testy, y_pred)
    r2 = r2_score(testy, y_pred)
    return pd.DataFrame([{
        'Model': model_name,
        'mse': mse, 
        'mae': mae, 
        'mae': mae, 
        'rmse': rmse, 
        'mape': mape, 
        'r2': r2
    }])

# Create a conv model

In [9]:
import os
from tensorflow.keras.preprocessing.image import load_img, img_to_array

2024-03-29 12:53:18.974370: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-03-29 12:53:19.026337: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-29 12:53:19.026385: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-29 12:53:19.027716: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-03-29 12:53:19.037500: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-03-29 12:53:19.038026: I tensorflow/core/platform/cpu_feature_guard.cc:1

## Prepare the data

In [10]:
def preprocess_image(image_path):
    img = load_img(image_path)
    
    img_array = img_to_array(img)
    
    return img_array


In [26]:
import numpy as np
images = []
target = data['CO(GT)']

# get all the images (i dont know too much about this process)
for folder in os.listdir('imgs'):
    for image in os.listdir(os.path.join('imgs', folder)):
        img = preprocess_image(os.path.join('imgs', folder, image))
        images.append(img)

images = np.array(images)
target = np.array(target)

In [27]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(images, target, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

## Define the model

In [28]:
import tensorflow as tf
from tensorflow.keras import layers, models

In [53]:
model = models.Sequential([
    layers.Input(shape=images.shape[1:]),
    layers.Conv2D(8, (5,5), activation='relu'),
    layers.Conv2D(8, (3,3), activation='relu'),
    layers.MaxPooling2D((2,2)),
    layers.Conv2D(16, (3,3), activation='relu'),
    layers.Conv2D(16, (2,2), activation='relu'),
    layers.Flatten(),
    layers.Dense(32, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.01)),
    layers.Dense(8, activation='sigmoid', kernel_regularizer=tf.keras.regularizers.l2(0.01)),
    layers.Dropout(0.2),
    layers.Dense(16, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.01)),
    layers.Dropout(0.2),
    layers.Dense(32, activation='elu', kernel_regularizer=tf.keras.regularizers.l2(0.01)),
    layers.Dropout(0.2),
    layers.Dense(16, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.01)),
    layers.Dense(8, activation='selu', kernel_regularizer=tf.keras.regularizers.l2(0.01)),
    layers.Dense(1)
])
model.compile(optimizer='adam', loss='mean_squared_error')
model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_val, y_val))

Epoch 1/50
131/131 [==============================] - 3s 10ms/step - loss: 2.8636 - val_loss: 2.1848
Epoch 2/50
131/131 [==============================] - 1s 9ms/step - loss: 2.2072 - val_loss: 2.0428
Epoch 3/50
131/131 [==============================] - 1s 9ms/step - loss: 2.0832 - val_loss: 1.9517
Epoch 4/50
131/131 [==============================] - 1s 8ms/step - loss: 1.9913 - val_loss: 1.8868
Epoch 5/50
131/131 [==============================] - 1s 9ms/step - loss: 1.9159 - val_loss: 1.8258
Epoch 6/50
131/131 [==============================] - 1s 10ms/step - loss: 1.8499 - val_loss: 1.7754
Epoch 7/50
131/131 [==============================] - 1s 11ms/step - loss: 1.8066 - val_loss: 1.7603
Epoch 8/50
131/131 [==============================] - 2s 12ms/step - loss: 1.7742 - val_loss: 1.7000
Epoch 9/50
131/131 [==============================] - 1s 11ms/step - loss: 1.7286 - val_loss: 1.6684
Epoch 10/50
131/131 [==============================] - 1s 11ms/step - loss: 1.7009 - val_loss: 

In [54]:
from sklearn.metrics import mean_squared_error
y_pred = model.predict(X_test)
results = pd.concat([results, appendError(model, y_test, y_pred)])

41/41 [==============================] - 0s 8ms/step


In [55]:
results

,Model,mse,mae,rmse,mape,r2
0,Sequential,1.366697,0.936868,1.169058,0.801673,0.003857
0,Sequential,1.372069,0.946211,1.171353,0.827817,-0.000058
